### Learning LangChain - Designing A Simple Agent

In [ ]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.utilities import GoogleSerperAPIWrapper

In [ ]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo"
temperature = 0.1
max_tokens = 1000

llm = ChatOpenAI(
    model = model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key = openai_api_key
)

In [ ]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplies two integers together"""
    
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Sums two integers together"""
    
    return first_int + second_int

@tool
def exponentize(first_int: int, second_int: int) -> int:
    """Exponentize the base to the exponent value"""
    
    return first_int ** second_int

@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    """
    
    search = GoogleSerperAPIWrapper()
    
    return search.run(query_string)

In [ ]:
tools = [add, multiply, exponentize, search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

In [ ]:
question = """
take 3 to the fifth power and multiply that result by the sum of twelve and three, then square the whole result, 
and also let me know the capital of Malaysia ...
and also need 5 bulleted points about Mahatma Gandhi Achievements
"""

response = agent_executor.invoke({
    "input": question
})

print(response)